In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("archive/Restaurants_Train_v2.csv")
df

,id,Sentence,Aspect Term,polarity,from,to
0,3121,But the staff was so horrible to us.,staff,negative,8,13
1,2777,"To be completely fair, the only redeeming fact...",food,positive,57,61
2,1634,"The food is uniformly exceptional, with a very...",food,positive,4,8
3,1634,"The food is uniformly exceptional, with a very...",kitchen,positive,55,62
4,1634,"The food is uniformly exceptional, with a very...",menu,neutral,141,145
...,...,...,...,...,...,...
3688,671,Each table has a pot of boiling water sunken i...,pot of boiling water,neutral,17,37
3689,671,Each table has a pot of boiling water sunken i...,meats,neutral,99,104
3690,671,Each table has a pot of boiling water sunken i...,vegetables,neutral,114,124
3691,671,Each table has a pot of boiling water sunken i...,rice,neutral,130,134


In [3]:
len(df.id.unique())

2021

In [4]:
df.drop_duplicates(subset=['id'], inplace=True, ignore_index=True)
df

,id,Sentence,Aspect Term,polarity,from,to
0,3121,But the staff was so horrible to us.,staff,negative,8,13
1,2777,"To be completely fair, the only redeeming fact...",food,positive,57,61
2,1634,"The food is uniformly exceptional, with a very...",food,positive,4,8
3,2846,"Not only was the food outstanding, but the lit...",food,positive,17,21
4,1458,Our agreed favorite is the orrechiete with sau...,orrechiete with sausage and chicken,positive,27,62
...,...,...,...,...,...,...
2016,2378,"The service was typical short-order, dinner type.",service,neutral,4,11
2017,1027,"We shared a bottle of sake, an order of edamam...",bottle of sake,neutral,12,26
2018,1735,I can't believe people complain about no chees...,cheese sticks,neutral,41,54
2019,777,"From the appetizers we ate, the dim sum and ot...",appetizers,positive,9,19


In [5]:
df.groupby(["polarity"]).count()

,id,Sentence,Aspect Term,from,to
polarity,,,,,
conflict,70,70,70,70,70
negative,436,436,436,436,436
neutral,301,301,301,301,301
positive,1214,1214,1214,1214,1214


In [6]:
support_set = df[df.polarity=="positive"].sample(n=8)
support_set = support_set.merge(df[df.polarity=="negative"].sample(n=8), how='outer')
support_set = support_set.merge(df[df.polarity=="neutral"].sample(n=8), how='outer')
support_set = support_set.merge(df[df.polarity=="conflict"].sample(n=8), how='outer')

support_set.head()

,id,Sentence,Aspect Term,polarity,from,to
0,3655,"I also ordered the Change Mojito, which was ou...",Change Mojito,positive,19,32
1,997,"This is such a lovely, peaceful place to eat o...",place,positive,32,37
2,957,"The menu consisted of standard brassiere food,...",menu,positive,4,8
3,1050,"Not only is the cuisine the best around, the s...",cuisine,positive,16,23
4,3291,Food is great.,Food,positive,0,4


In [7]:
#removing the sup set from the data
print("before removing: ", len(df))
for id in support_set.id:
  index = df[df.id==id].index[0]
  df.drop([index], inplace=True)
print("after removing: ", len(df))

before removing:  2021
after removing:  1989


In [8]:
support_set.drop(columns=['Aspect Term','from','to'], inplace=True)

In [9]:
support_set.head()

,id,Sentence,polarity
0,3655,"I also ordered the Change Mojito, which was ou...",positive
1,997,"This is such a lovely, peaceful place to eat o...",positive
2,957,"The menu consisted of standard brassiere food,...",positive
3,1050,"Not only is the cuisine the best around, the s...",positive
4,3291,Food is great.,positive


In [10]:
query_set = df[df.polarity=="positive"].sample(n=2)
query_set = query_set.merge(df[df.polarity=="negative"].sample(n=2), how='outer')
query_set = query_set.merge(df[df.polarity=="neutral"].sample(n=2), how='outer')
query_set = query_set.merge(df[df.polarity=="conflict"].sample(n=2), how='outer')

query_set.drop(columns=['Aspect Term','from','to'], inplace=True)
query_set

,id,Sentence,polarity
0,3135,You can also special order any kind of dumplin...,positive
1,2434,This is an amazing place to try some roti rolls.,positive
2,1795,It was like the servers forgot that they actua...,negative
3,2194,They don't seem to place an emphasis on specia...,negative
4,1462,Appetizers are somewhere around $7 each and th...,neutral
5,3554,"if this happens, just ask for real naan.",neutral
6,3355,- for dessert we split chocolate cake and vani...,conflict
7,661,There is usually a wait but it is well worth it.,conflict


In [11]:
sup_set_labels = support_set.polarity.unique()
labels = {label:i+1 for i,label in enumerate(sup_set_labels)}
labels

{'positive': 1, 'negative': 2, 'neutral': 3, 'conflict': 4}

In [13]:
def tokenize_inputs(sentences,labels):
  input_ids = []
  attention_masks = []

  for sent in sentences:

    encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                        )
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  return input_ids.type(torch.float32).squeeze(1), attention_masks.type(torch.float32), torch.tensor(labels)

#sup_input_ids, sup_attention_masks, sup_labels = tokenize_inputs(support_set.Sentence,support_set.polarity)
#qry_input_ids, qry_attention_masks, qry_labels = tokenize_inputs(query_set.Sentence,query_set.polarity)


In [14]:
from torch import nn

def cosine_sim(qry_input_ids,sup_input_ids):
  sim_scores = []
  cos = nn.CosineSimilarity()
  for qry_row in qry_input_ids:
    avg_cls_score = 0
    for sup_row in sup_input_ids:
      #print(sup_row.shape)
      #sim_scores.append(cos(qry_row.unsqueeze(0),sup_row.unsqueeze(0)))
      #print(qry_row)
      avg_cls_score += cos(qry_row.unsqueeze(0),sup_row.unsqueeze(0))

    sim_scores.append(avg_cls_score / qry_input_ids.shape[0])
  return sim_scores

#sim_scores = cosine_sim(qry_input_ids,sup_input_ids)
#torch.tensor(sim_scores).shape

In [15]:
val_sup = df[df.polarity=="positive"].sample(n=8)
val_sup = val_sup.merge(df[df.polarity=="negative"].sample(n=8), how='outer')
val_sup = val_sup.merge(df[df.polarity=="neutral"].sample(n=8), how='outer')
val_sup = val_sup.merge(df[df.polarity=="conflict"].sample(n=8), how='outer')

val_qry = df[df.polarity=="positive"].sample(n=2)
val_qry = val_qry.merge(df[df.polarity=="negative"].sample(n=2), how='outer')
val_qry = val_qry.merge(df[df.polarity=="neutral"].sample(n=2), how='outer')
val_qry = val_qry.merge(df[df.polarity=="conflict"].sample(n=2), how='outer')

In [17]:
def prepare_inputs(input1,label,device):
    label=label.to(device)
    mask=input1['attention_mask'].to(device)
    input_id=input1['input_ids'].squeeze(1).to(device)
    return (input_id,mask,label)

In [18]:
import torch
import numpy as np
from transformers import BertTokenizer, DistilBertModel, DistilBertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self,df,labels,column='Sentence'):
        self.labels=[labels[label] for label in df['polarity']]
        self.texts=[tokenizer(text.lower(),padding='max_length',max_length=64,truncation=True,return_tensors="pt") for text in df[column]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self,idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self,idx):
        return self.texts[idx]

    def __getitem__(self,idx):
        batch_texts=self.get_batch_texts(idx)
        batch_y=self.get_batch_labels(idx)

        return batch_texts,batch_y


support_data=Dataset(support_set,labels)
query_data=Dataset(query_set,labels)
val_support_data=Dataset(val_sup,labels)
val_query_data=Dataset(val_qry,labels)

In [42]:
from torch import nn
from transformers import BertModel, BertForSequenceClassification

class BertClassifier(nn.Module):

    def __init__(self,dropout=0.5,num_classes=4):

        super(BertClassifier,self).__init__()

        self.bert=BertModel.from_pretrained('bert-base-uncased')
        self.dropout=nn.Dropout(dropout)
        self.linear1=nn.Linear(768,512)
        self.linear2=nn.Linear(512,4)
        self.relu=nn.ReLU()
        self.softmax=nn.Softmax(dim=1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,input_id,mask):

        last_hidden_layer,pooler_output = self.bert(input_ids=input_id,attention_mask=mask,return_dict=False)
        
        dropout_output=self.dropout(pooler_output)
        linear_output=self.linear1(dropout_output)
        linear_output=self.relu(linear_output)

        dropout_output=self.dropout(linear_output)
        linear_output=self.linear2(dropout_output)
        final_layer=self.softmax(linear_output)
        
        return final_layer

In [43]:
from torch.optim import Adam, AdamW
from tqdm import tqdm

def train(model, support, query, val_sup, val_qry, learning_rate, epochs):

    support_data=torch.utils.data.DataLoader(support, batch_size=2)
    query_data=torch.utils.data.DataLoader(query, batch_size=2)
    val_support_data=torch.utils.data.DataLoader(val_sup, batch_size=2)
    val_query_data=torch.utils.data.DataLoader(val_qry, batch_size=2)

    use_cuda=torch.cuda.is_available()
    device=torch.device("cuda" if use_cuda else "cpu")

    criterion=nn.CrossEntropyLoss()
    optimizer=AdamW(model.parameters(),lr=learning_rate)
    cos = nn.CosineSimilarity()


    if use_cuda:
        model=model.cuda()
        criterion=criterion.cuda()

    for epoch_num in range(epochs):
        model.train()

        for qry_inp, qry_lab in query_data:
          qry_input_id, qry_mask, qry_label = prepare_inputs(qry_inp,qry_lab,device)
          for sup_inp, sup_lab in support_data:
            sup_input_id, sup_mask, sup_label = prepare_inputs(sup_inp,sup_lab,device)

            sup_output = model(sup_input_id, sup_mask)
            qry_output = model(qry_input_id, qry_mask)

            #print("Query output: ",qry_output)
            #print("Support output: ",sup_output)
            logits = cos(qry_output, sup_output)
            #print("cosine sim scores: ", logits)
            #print(logits)
            #print(qry_label.float().unsqueeze(1).softmax(dim=1))
            loss = criterion(logits, qry_label.float())
            #print(logits, qry_label)
            #print(loss)
            model.zero_grad()
            loss.backward()

            optimizer.step()


        model.eval()
        with torch.no_grad():
          for qry_inp, qry_lab in val_query_data:
            qry_input_id, qry_mask, qry_label = prepare_inputs(qry_inp,qry_lab,device)
            for sup_inp, sup_lab in val_support_data:
              sup_input_id, sup_mask, sup_label = prepare_inputs(sup_inp,sup_lab,device)

              sup_output = model(sup_input_id, sup_mask)
              qry_output = model(qry_input_id, qry_mask)

              logits = cos(qry_output, sup_output)

              loss = criterion(logits, qry_label.float())
              print(qry_output, qry_label)

In [44]:
EPOCHS=1
model=BertClassifier()
#model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4, problem_type="multi_label_classification")
LR=2e-5

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
train(model,support_data,query_data,val_support_data,val_query_data,LR,EPOCHS)

tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]], device='cuda:0') tensor([1, 1], device='cuda:0', dtype=torch.int32)
tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]], device='cuda:0') tensor([1, 1], device='cuda:0', dtype=torch.int32)
tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]], device='cuda:0') tensor([1, 1], device='cuda:0', dtype=torch.int32)
tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]], device='cuda:0') tensor([1, 1], device='cuda:0', dtype=torch.int32)
tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]], device='cuda:0') tensor([1, 1], device='cuda:0', dtype=torch.int32)
tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]], device='cuda:0') tensor([1, 1], device='cuda:0', dtype=torch.int32)
tensor([[0.2454, 0.2510, 0.2359, 0.2677],
        [0.2464, 0.2478, 0.2384, 0.2674]

        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device='cuda:0', dtype=torch.int32)
tensor([[0.2487, 0.2479, 0.2330, 0.2704],
        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device='cuda:0', dtype=torch.int32)
tensor([[0.2487, 0.2479, 0.2330, 0.2704],
        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device='cuda:0', dtype=torch.int32)
tensor([[0.2487, 0.2479, 0.2330, 0.2704],
        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device='cuda:0', dtype=torch.int32)
tensor([[0.2487, 0.2479, 0.2330, 0.2704],
        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device='cuda:0', dtype=torch.int32)
tensor([[0.2487, 0.2479, 0.2330, 0.2704],
        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device='cuda:0', dtype=torch.int32)
tensor([[0.2487, 0.2479, 0.2330, 0.2704],
        [0.2467, 0.2520, 0.2336, 0.2677]], device='cuda:0') tensor([4, 4], device=

In [ ]:
torch.tensor(val_qry.polarity.map(lambda x: labels[x])).float()

In [ ]:
soft=nn.Softmax()
soft(torch.tensor(val_qry.polarity.map(lambda x: labels[x])).float())

In [48]:
soft=nn.Softmax()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_data=Dataset(df.head(),labels)
for inp,lab in torch.utils.data.DataLoader(test_data):
  input_id, mask, label = prepare_inputs(inp,lab,device)
  pred=model(input_id,mask)
  print(pred, label)

tensor([[0.2427, 0.2522, 0.2356, 0.2695]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([2], device='cuda:0', dtype=torch.int32)
tensor([[0.2439, 0.2482, 0.2389, 0.2690]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([1], device='cuda:0', dtype=torch.int32)
tensor([[0.2523, 0.2458, 0.2322, 0.2697]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([1], device='cuda:0', dtype=torch.int32)
tensor([[0.2460, 0.2514, 0.2333, 0.2693]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([1], device='cuda:0', dtype=torch.int32)
tensor([[0.2499, 0.2469, 0.2360, 0.2672]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([1], device='cuda:0', dtype=torch.int32)


In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
soft=nn.Softmax()

soft(torch.tensor([[ 0.3172,  0.4676,  0.9003, -0.0454],
        [-0.0521,  0.3826,  1.0570, -0.0943],
        [-0.0188,  0.4739,  0.7390, -0.2135],
        [ 0.2028,  0.5530,  0.7879, -0.1929],
        [ 0.1532,  0.3444,  0.6048, -0.4445],
        [ 0.1819,  0.2178,  0.5018, -0.0368],
        [ 0.2170,  0.4156,  0.6320, -0.0466],
        [-0.0087,  0.5085,  0.9123, -0.0525]]))

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

la = df.head().polarity.map(lambda x: labels[x])

test_ids, test_masks, test_labels = tokenize_inputs(df.head().Sentence, la)

test_ids.to(device)
test_masks.to(device)
test_labels.to(device)

In [ ]:
preds = model(test_ids.long().to(device), test_masks.long().to(device))
preds

In [ ]:
test_labels

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
print(input)
print(target)

In [ ]:
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()
print(input)
print(target)